# 일대다 병합
- 데이터 구조가 너무 많이 바뀔 수 있으므로 유의한다
1. 어느 열을 병합 기준 열로 삼는 것이 타당한지 확인(병합 기준 열이 반드시 같은 열일 필요는 없음)
2. 어떤 종류의 조인을 사용할지와 그 이유를 알아야한다!
3. 양쪽 데이터테이블에 행이 몇개씩 있는지 확인
4. 조인 유형, 각 데이터셋의 행 수, 사전 조사한 병합 기준 값의 개수에 따라 몇 개의 행이 유지될 지를 예상해야한다.

### 데이터 : GHCN 기상 관측소 데이터 
- 각 국가에 대응하는 행이 있는 데이터프레임과
- 기상관측소에 대응하는 행이 있는 데이터프레임 사용(나라마다 여러 기상관측소 있음)
    - 국가별 데이터에 대해 일대다 병합을 수행해, 한 행에 하나의 국가가 오게 하고, 국가별로 여러 관측소가 오게 한다.

In [4]:
import pandas as pd
countries = pd.read_csv('data/ltcountries.csv')
locations = pd.read_csv('data/ltlocations.csv')

In [5]:
countries.set_index('countryid', inplace = True)
locations.set_index('countryid', inplace = True)

In [7]:
countries.head()

,country
countryid,
AC,Antigua and Barbuda
AE,United Arab Emirates
AF,Afghanistan
AG,Algeria
AJ,Azerbaijan


In [10]:
countries.index.nunique() == countries.shape[0]

True

In [12]:
locations[['locationid', 'latitude', 'stnelev']].head(10)

,locationid,latitude,stnelev
countryid,,,
AC,ACW00011604,57.7667,18.0
AE,AE000041196,25.3330,34.0
AE,AEM00041184,25.6170,31.0
AE,AEM00041194,25.2550,10.4
AE,AEM00041216,24.4300,3.0
AE,AEM00041217,24.4330,26.8
AE,AEM00041218,24.2620,264.9
AF,AF000040930,35.3170,3366.0
AF,AFM00040911,36.7000,378.0


### join 으로 국가 및 관측소 위치에 대해 왼쪽 조인 수행

In [13]:
stations = countries.join(locations)

In [14]:
stations[['locationid', 'latitude', 'stnelev', 'country']].head(10)

,locationid,latitude,stnelev,country
countryid,,,,
AC,ACW00011604,57.7667,18.0,Antigua and Barbuda
AE,AE000041196,25.3330,34.0,United Arab Emirates
AE,AEM00041184,25.6170,31.0,United Arab Emirates
AE,AEM00041194,25.2550,10.4,United Arab Emirates
AE,AEM00041216,24.4300,3.0,United Arab Emirates
AE,AEM00041217,24.4330,26.8,United Arab Emirates
AE,AEM00041218,24.2620,264.9,United Arab Emirates
AF,AF000040930,35.3170,3366.0,Afghanistan
AF,AFM00040911,36.7000,378.0,Afghanistan


### 병합 기준열의 일치 여부 확인
- checkmerge 함수 실행

In [15]:
countries = pd.read_csv('data/ltcountries.csv')
locations = pd.read_csv('data/ltlocations.csv')

In [23]:
def checkmerge(dfleft, dfright, idvar):
    dfleft['inleft'] = "Y"
    dfright['inright'] = "Y"
    dfboth = pd.merge(dfleft[[idvar, 'inleft']], dfright[[idvar, 'inright']], on=[idvar], how='outer')
    dfboth.fillna('N', inplace=True)
    print(pd.crosstab(dfboth.inleft, dfboth.inright))
    print(dfboth.loc[(dfboth.inleft=="N")|(dfboth.inright=='N')])

In [25]:
checkmerge(countries.copy(), locations.copy(), 'countryid')

inright  N      Y
inleft           
N        0      1
Y        2  27472
      countryid inleft inright
9715         LQ      Y       N
13103        ST      Y       N
27474        FO      N       Y


### 한쪽 파일에만 있는 행 표시

In [26]:
countries.loc[(countries.countryid.isin(["LQ", "ST"]))]

,countryid,country
124,LQ,Palmyra Atoll [United States]
195,ST,Saint Lucia


In [27]:
locations.loc[(locations.countryid=="FO")]

,locationid,latitude,longitude,stnelev,station,countryid
7363,FOM00006009,61.4,-6.667,102.0,AKRABERG,FO


### locations와 countries 데이터프레임 병합

In [28]:
stations = pd.merge(countries, locations, on=['countryid'], how='left')

In [29]:
stations[['locationid', 'latitude', 'stnelev', 'country']].head(10)

,locationid,latitude,stnelev,country
0,ACW00011604,57.7667,18.0,Antigua and Barbuda
1,AE000041196,25.3330,34.0,United Arab Emirates
2,AEM00041184,25.6170,31.0,United Arab Emirates
3,AEM00041194,25.2550,10.4,United Arab Emirates
4,AEM00041216,24.4300,3.0,United Arab Emirates
5,AEM00041217,24.4330,26.8,United Arab Emirates
6,AEM00041218,24.2620,264.9,United Arab Emirates
7,AF000040930,35.3170,3366.0,Afghanistan
8,AFM00040911,36.7000,378.0,Afghanistan
9,AFM00040938,34.2100,977.2,Afghanistan


In [30]:
stations.shape

(27474, 7)

In [31]:
stations.loc[stations.countryid.isin(["LQ", "ST"])].isnull().sum()

countryid     0
country       0
locationid    2
latitude      2
longitude     2
stnelev       2
station       2
dtype: int64

In [32]:
stations.loc[stations.countryid.isin(["LQ", "ST"])]

,countryid,country,locationid,latitude,longitude,stnelev,station
9715,LQ,Palmyra Atoll [United States],NaN,NaN,NaN,NaN,NaN
13103,ST,Saint Lucia,NaN,NaN,NaN,NaN,NaN
